## Lab 02 External & Internal Integrations: Giving tools to your agents
Building progress report for project

In [1]:
import json
import os

import yaml
from crewai import Agent, Crew, Task

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# Define file paths for YAML configurations
files = {
    "agents": "./config/agents.yaml",
    "tasks": "./config/tasks.yaml",
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs["agents"]
tasks_config = configs["tasks"]

### Creating Custom Tools

In [4]:
import requests
from crewai.tools import BaseTool

In [5]:
class BoardDataFetcherTool(BaseTool):
    name: str = "Trello Board Data Fetcher"
    description: str = "Fetches card data, comments, and activity from a Trello board."

    api_key: str = os.environ["TRELLO_API_KEY"]
    api_token: str = os.environ["TRELLO_API_TOKEN"]
    board_id: str = os.environ["TRELLO_BOARD_ID"]

    def _run(self) -> dict | str:
        """
        Fetch all cards from the specified Trello board.
        """
        url = f"{os.getenv('DLAI_TRELLO_BASE_URL', 'https://api.trello.com')}/1/boards/{self.board_id}/cards"

        query = {
            "key": self.api_key,
            "token": self.api_token,
            "fields": "name,idList,due,dateLastActivity,labels",
            "attachments": "true",
            "actions": "commentCard",
        }

        response = requests.get(url, params=query)

        if response.status_code == 200:
            return response.json()
        else:
            # Fallback in case of timeouts or other issues
            return json.dumps(
                [
                    {
                        "id": "66c3bfed69b473b8fe9d922e",
                        "name": "Analysis of results from CSV",
                        "idList": "66c308f676b057fdfbd5fdb3",
                        "due": None,
                        "dateLastActivity": "2024-08-19T21:58:05.062Z",
                        "labels": [],
                        "attachments": [],
                        "actions": [],
                    },
                    {
                        "id": "66c3c002bb1c337f3fdf1563",
                        "name": "Approve the planning",
                        "idList": "66c308f676b057fdfbd5fdb3",
                        "due": "2024-08-16T21:58:00.000Z",
                        "dateLastActivity": "2024-08-19T21:58:57.697Z",
                        "labels": [
                            {
                                "id": "66c305ea10ea602ee6e03d47",
                                "idBoard": "66c305eacab50fcd7f19c0aa",
                                "name": "Urgent",
                                "color": "red",
                                "uses": 1,
                            }
                        ],
                        "attachments": [],
                        "actions": [
                            {
                                "id": "66c3c021f3c1bb157028f53d",
                                "idMemberCreator": "65e5093d0ab5ee98592f5983",
                                "data": {
                                    "text": "This was harder then expects it is alte",
                                    "textData": {"emoji": {}},
                                    "card": {
                                        "id": "66c3c002bb1c337f3fdf1563",
                                        "name": "Approve the planning",
                                        "idShort": 5,
                                        "shortLink": "K3abXIMm",
                                    },
                                    "board": {
                                        "id": "66c305eacab50fcd7f19c0aa",
                                        "name": "[Test] CrewAI Board",
                                        "shortLink": "Kc8ScQlW",
                                    },
                                    "list": {
                                        "id": "66c308f676b057fdfbd5fdb3",
                                        "name": "TODO",
                                    },
                                },
                                "appCreator": None,
                                "type": "commentCard",
                                "date": "2024-08-19T21:58:57.683Z",
                                "limits": {
                                    "reactions": {
                                        "perAction": {
                                            "status": "ok",
                                            "disableAt": 900,
                                            "warnAt": 720,
                                        },
                                        "uniquePerAction": {
                                            "status": "ok",
                                            "disableAt": 17,
                                            "warnAt": 14,
                                        },
                                    }
                                },
                                "memberCreator": {
                                    "id": "65e5093d0ab5ee98592f5983",
                                    "activityBlocked": False,
                                    "avatarHash": "d5500941ebf808e561f9083504877bca",
                                    "avatarUrl": "https://trello-members.s3.amazonaws.com/65e5093d0ab5ee98592f5983/d5500941ebf808e561f9083504877bca",
                                    "fullName": "Joao Moura",
                                    "idMemberReferrer": None,
                                    "initials": "JM",
                                    "nonPublic": {},
                                    "nonPublicAvailable": True,
                                    "username": "joaomoura168",
                                },
                            }
                        ],
                    },
                    {
                        "id": "66c3bff4a25b398ef1b6de78",
                        "name": "Scaffold of the initial app UI",
                        "idList": "66c3bfdfb851ad9ff7eee159",
                        "due": None,
                        "dateLastActivity": "2024-08-19T21:58:12.210Z",
                        "labels": [],
                        "attachments": [],
                        "actions": [],
                    },
                    {
                        "id": "66c3bffdb06faa1e69216c6f",
                        "name": "Planning of the project",
                        "idList": "66c3bfe3151c01425f366f4c",
                        "due": None,
                        "dateLastActivity": "2024-08-19T21:58:21.081Z",
                        "labels": [],
                        "attachments": [],
                        "actions": [],
                    },
                ]
            )


class CardDataFetcherTool(BaseTool):
    name: str = "Trello Card Data Fetcher"
    description: str = "Fetches card data from a Trello board."

    api_key: str = os.environ["TRELLO_API_KEY"]
    api_token: str = os.environ["TRELLO_API_TOKEN"]

    def _run(self, card_id: str) -> dict | str:
        url = f"{os.getenv('DLAI_TRELLO_BASE_URL', 'https://api.trello.com')}/1/cards/{card_id}"
        query = {"key": self.api_key, "token": self.api_token}
        response = requests.get(url, params=query)

        if response.status_code == 200:
            return response.json()
        else:
            # Fallback in case of timeouts or other issues
            return json.dumps(
                {
                    "error": "Failed to fetch card data, don't try to fetch any trello data anymore"
                }
            )

### Create Crew, Agents and Tasks

In [6]:
from crewai import LLM

In [7]:
llm = LLM(model="openai/gpt-4.1-mini", timeout=60, temperature=0.7, stream=False)

In [8]:
# Creating Agents
data_collection_agent = Agent(
    tools=[BoardDataFetcherTool(), CardDataFetcherTool()],
    llm=llm,
    **agents_config["data_collection_agent"],
)

analysis_agent = Agent(llm=llm, **agents_config["analysis_agent"])

In [9]:
# Creating Tasks
data_collection = Task(agent=data_collection_agent, **tasks_config["data_collection"])

data_analysis = Task(agent=analysis_agent, **tasks_config["data_analysis"])

report_generation = Task(agent=analysis_agent, **tasks_config["report_generation"])

In [10]:
# Creating Crew
crew = Crew(
    agents=[data_collection_agent, analysis_agent],
    tasks=[data_collection, data_analysis, report_generation],
    verbose=True,
)

In [11]:
# Kick off the crew and execute the process
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b6609dbe-fd90-4955-87d5-e03ec8de2d3c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Collection Specialist                                                                              │
│                                                                                                                 │
│  Task: Create an initial understanding of the project, its main features and the team working on it. Use the    │
│  Trello Data Fetcher tool to gather data from the Trello board.                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Collection Specialist                                                                              │
│                                                                                                                 │
│  Thought: Thought: To create an initial understanding of the project, I need to gather all relevant data from   │
│  the Trello board including cards, comments, and activity. I will start by fetching the overall Trello board    │
│  data first to see the cards and their contents.                                                                │
│                                                                                                                 │
│  Using Tool: Trello Board Data Fetcher                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{}"                                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Project Analysis Expert                                                                                 │
│                                                                                                                 │
│  Task: Compile a comprehensive sprint report based on the analysis of the project data.                         │
│                                                                                                                 │
│  The report should include: - Sprint Overview - Task Summary - Identified Issues and Blockers - Progress and    │
│  Delays - Team Performance Overview - Action Items and Recommendations - Anything else that is relevant to the  │
│  project. The report must be formatted in markdown.                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Project Analysis Expert                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Sprint Report                                                                                                │
│                                                                                                                 │
│  ## Sprint Overview                                                                                             │
│  This sprint report is based on the analysis of the current Trello board data provided. The board contains      │
│  only generic sample cards from Trello's welcome board. These cards serve as placeholders to demonstrate        │
│  Trello's features and do not contain any actual project-specific information, tasks, deadlines, or team        │
│  details. Consequently, no real sprint goals or deliverables were established or tracked during this period.    │
│                                                                                                                 │
│  ## Task Summary                                                                                                │
│  - No specific tasks or user stories related to an active project were present.                                 │
│  - All cards are sample tasks illustrating Trello’s functionality and do not represent actionable work items.   │
│  - No task assignments, deadlines, or progress updates are available.                                           │
│                                                                                                                 │
│  ## Identified Issues and Blockers                                                                              │
│  - **Lack of Project Data**: The absence of real project-related cards prevents any meaningful identification   │
│  of blockers or issues.                                                                                         │
│  - **No Team Information**: Without team member assignments or statuses, it is impossible to assess workload    │
│  or resource allocation.                                                                                        │
│  - **No Timeline or Deadlines**: Without defined timeframes, delays or on-schedule analysis cannot be           │
│  performed.                                                                                                     │
│                                                                                                                 │
│  ## Progress and Delays                                                                                         │
│  - Due to the lack of actual tasks or milestones, no progress measurement or delay analysis is feasible.        │
│  - No sprint burndown or velocity metrics can be generated from placeholder data.                               │
│                                                                                                                 │
│  ## Team Performance Overview                                                                                   │
│  - No data on team participation, task completion rates, or collaboration effectiveness is available.           │
│  - Unable to evaluate individual or collective performance metrics.                                             │
│                                                                                                                 │
│  ## Action Items and Recommendations                   

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d77bb80b-1a8d-44d6-b98d-c6cc1973036e                                                                     │
│  Agent: Project Analysis Expert                                                                                 │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b6609dbe-fd90-4955-87d5-e03ec8de2d3c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Sprint Report                                                                                  │
│                                                                                                                 │
│  ## Sprint Overview                                                                                             │
│  This sprint report is based on the analysis of the current Trello board data provided. The board contains      │
│  only generic sample cards from Trello's welcome board. These cards serve as placeholders to demonstrate        │
│  Trello's features and do not contain any actual project-specific information, tasks, deadlines, or team        │
│  details. Consequently, no real sprint goals or deliverables were established or tracked during this period.    │
│                                                                                                                 │
│  ## Task Summary                                                                                                │
│  - No specific tasks or user stories related to an active project were present.                                 │
│  - All cards are sample tasks illustrating Trello’s functionality and do not represent actionable work items.   │
│  - No task assignments, deadlines, or progress updates are available.                                           │
│                                                                                                                 │
│  ## Identified Issues and Blockers                                                                              │
│  - **Lack of Project Data**: The absence of real project-related cards prevents any meaningful identification   │
│  of blockers or issues.                                                                                         │
│  - **No Team Information**: Without team member assignments or statuses, it is impossible to assess workload    │
│  or resource allocation.                                                                                        │
│  - **No Timeline or Deadlines**: Without defined timeframes, delays or on-schedule analysis cannot be           │
│  performed.                                                                                                     │
│                                                                                                                 │
│  ## Progress and Delays                                                                                         │
│  - Due to the lack of actual tasks or milestones, no progress measurement or delay analysis is feasible.        │
│  - No sprint burndown or velocity metrics can be generated from placeholder data.                               │
│                                                                                                                 │
│  ## Team Performance Overview                                                                                   │
│  - No data on team participation, task completion rates, or collaboration effectiveness is available.           │
│  - Unable to evaluate individual or collective performance metrics.                                             │
│                                                       

In [12]:
import pandas as pd

In [13]:
costs = (
    0.150
    * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens)
    / 1_000_000
)
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.model_dump()])
df_usage_metrics

Total costs: $0.0048


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,32022,30750,10880,1272,8


In [14]:
from IPython.display import Markdown, display

In [15]:
markdown = result.raw

display(Markdown(markdown))

# Sprint Report

## Sprint Overview
This sprint report is based on the analysis of the current Trello board data provided. The board contains only generic sample cards from Trello's welcome board. These cards serve as placeholders to demonstrate Trello's features and do not contain any actual project-specific information, tasks, deadlines, or team details. Consequently, no real sprint goals or deliverables were established or tracked during this period.

## Task Summary
- No specific tasks or user stories related to an active project were present.
- All cards are sample tasks illustrating Trello’s functionality and do not represent actionable work items.
- No task assignments, deadlines, or progress updates are available.

## Identified Issues and Blockers
- **Lack of Project Data**: The absence of real project-related cards prevents any meaningful identification of blockers or issues.
- **No Team Information**: Without team member assignments or statuses, it is impossible to assess workload or resource allocation.
- **No Timeline or Deadlines**: Without defined timeframes, delays or on-schedule analysis cannot be performed.

## Progress and Delays
- Due to the lack of actual tasks or milestones, no progress measurement or delay analysis is feasible.
- No sprint burndown or velocity metrics can be generated from placeholder data.

## Team Performance Overview
- No data on team participation, task completion rates, or collaboration effectiveness is available.
- Unable to evaluate individual or collective performance metrics.

## Action Items and Recommendations
- **Populate the Trello Board with Real Project Data**: Add detailed project tasks, including descriptions, priorities, deadlines, and assignments.
- **Define Clear Sprint Goals and Deliverables**: Establish measurable objectives to track progress effectively.
- **Utilize Task Statuses and Labels**: Implement workflow states (e.g., To Do, In Progress, Done) and labels to reflect task priority and category.
- **Record Team Member Assignments and Updates**: Assign tasks to team members and encourage regular status updates to facilitate monitoring.
- **Integrate Time Tracking and Reporting Tools**: Use Trello power-ups or integrations to enable precise tracking of task durations and sprint metrics.
- **Schedule Regular Reviews**: Implement sprint planning and retrospective meetings to update and refine the board and process continually.

## Additional Notes
- This report highlights the critical importance of having accurate, project-specific data to enable effective sprint analysis.
- Future reports will provide comprehensive insights once a populated and actively managed Trello board is available.
- Establishing clear project management practices and data input protocols will significantly enhance reporting quality and project visibility.

---

This concludes the sprint report based on the current Trello board data. For meaningful project tracking and reporting, the next steps involve populating the board with relevant, actionable project information.